In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

%load_ext version_information
%version_information deap, matplotlib, numpy, pandas, seaborn, sklearn

Software versions
Python 3.5.1 64bit [GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
IPython 4.1.2
OS Linux 3.13.0 62 generic x86_64 with debian jessie sid
deap 1.0
matplotlib 1.5.1
numpy 1.10.4
pandas 0.18.0
seaborn 0.7.0
sklearn 0.17.1
Sun May 08 19:42:00 2016 BST

In [2]:
%matplotlib inline

from IPython.display import display
from ipywidgets import widgets
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import OrderedDict as od
import random
import re
import seaborn as sns

#pd.set_option('html', False)
np.set_printoptions(threshold=np.nan)
sns.set()

# Transform collected trade data into proper format and export it

In [137]:
path = '../btc-data/BTC_Trades_raw.csv'
date_parse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')

data = pd.read_csv(path)

data.TradeData = data.TradeData.map(lambda x: float(re.sub('[^0-9,.]', '', x)))
data.TradeID = data.TradeID.map(lambda x: float(re.sub('[^0-9,.]', '', x)))

data.rename(columns = {data.columns[0]: 'Amount', data.columns[1]: 'Price'}, inplace = True)
data.insert(0, 'Trade ID', data.index)

data['Date'] = data['Date'].map(date_parse)

data.set_index('Date', inplace = True)
data.sort_values(by = 'Trade ID', inplace = True)

data.to_csv(path_or_buf='../btc-data/BTC_Trades_clean.csv')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:15: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


# Transform collected order book data into proper format

In [15]:
path = '../btc-data/BTC_OB_raw.csv'
date_parse = lambda x: pd.datetime.strptime(x, '%d/%m/%Y %H:%M:%S')

data = pd.read_csv(path, sep='",')

data.rename(columns = {data.columns[0]: 'LOB Data', 
                       data.columns[1]: 'ID', 
                       data.columns[2]: 'Date'}, inplace = True)

data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('asks', 'asks ', x))
data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('bids', 'bids ', x))
data['LOB Data'] = data['LOB Data'].map(lambda x: re.sub('[^0-9,.,asks ,bids ,]', '', x))
data['ID'] = data['ID'].map(lambda x: re.sub('\D', '', x))
data['Date'] = data['Date'].map(lambda x: re.sub('[^0-9,/,:, ]', '', x))

data['Date'] = data['Date'].map(date_parse)

bids = data['LOB Data'].map(lambda x: re.split(',', re.sub('bids ', '', re.sub('^asks [0-9,.]*', '', x))))
asks = data['LOB Data'].map(lambda x: re.split(',', re.sub('asks ', '', re.sub('bids [0-9,.]*', '', x)[:-1])))

data.insert(1, 'Bids', bids)
data.insert(1, 'Asks', asks)

data['Bids'] = data['Bids'].map(lambda x: list(zip(x[::2], x[1::2])))
data['Asks'] = data['Asks'].map(lambda x: list(zip(x[::2], x[1::2])))

data.set_index('Date', inplace = True)
data.drop('LOB Data', axis = 1, inplace = True)

data = data[['ID','Asks', 'Bids']]
data['ID'] = data['ID'].map(lambda x: int(x))
data['Asks'] = data['Asks'].map(lambda x: list(map(lambda t: (float(t[0]), float(t[1])), x)))
data['Bids'] = data['Bids'].map(lambda x: list(map(lambda t: (float(t[0]), float(t[1])), x)))
data.sort_values(by = 'ID', inplace = True)

#data.to_csv(path_or_buf='../btc-data/BTC_OB_clean_test.csv')

/home/dl3913/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.


# Recreate the order book - NOT WORKING

In [129]:
askside = [ (373.1,0.53671),
            (373.23,17.24),
            (373.27,7.9555),
            (373.28,1.3363),
            (373.29,0.11771),
            (373.5,55.04631),
            (373.55,10.6796),
            (373.56,1.43616041),
            (373.59,0.26566839),
            (373.6,1.43882788),
            (373.61,1.43841224),
            (373.65,0.2630484),
            (373.71,1.69908455),
            (373.73,0.43790512),
            (373.75,1.54945824),
            (373.77,1.85661036),
            (373.78,0.17786362),
            (373.8,2.76357342),
            (373.81,1.73747649),
            (373.82,1.16956076)
]
bidside = [(372.85,2.61805646),
           (372.84,8.9237),
           (372.67,8.0285),
           (372.66,3.28509036),
           (372.6,0.02326771),
           (372.25,10.6798),
           (372.24,6.493),
           (372.2,2.0),
           (371.96,6.692),
           (371.74,0.04),
           (371.73,7.265),
           (371.7,0.02231636),
           (371.49,6.928),
           (371.4,0.02233438),
           (371.28,7.441),
           (371.1,0.02235244),
           (371.04,6.84),
           (370.9,0.262),
           (370.88,2.16480323),
           (370.86,45.75)
]


#print(data.tail())
#data.sort_values(by = 'ID', inplace = True)
#print(data.ix[5, 'Asks'])
    
for i in range(int(len(data)/100)):
    #print('Asks: ', askside)
    #print('To update: ', asks)
    
    for ask in data.ix[i, 'Asks']:
        found = False
        for j in range(len(askside)):
            price = askside[j][0]
            vol = askside[j][1]
        if not(found):
            if price == ask[0]:
                if ask[1] == 0:
                    del askside[j]
                else:
                    del askside[j]
                    askside.insert(j, ask)
                found = True
            elif price < ask[0]:
                if ask[1] > 0:
                    askside.insert(j, ask)
                found = True

    #while len(askside) > 20:
        #askside.remove(askside[-1])
            
    for bid in data.ix[i, 'Bids']:
        found = False
        for j in range(len(bidside)):
            price = bidside[j][0]
            vol = bidside[j][1]
        if not(found):
            if price == bid[0]:
                if bid[1] == 0:
                    del bidside[j]
                else:
                    del bidside[j]
                    bidside.insert(j, bid)
                found = True
            elif price > bid[0]:
                if bid[1] > 0:
                    bidside.insert(j, bid)
                found = True

    #while len(bidside) > 20:
        #bidside.remove(bidside[-1])

[(570.0, 2.23492593), (376.32, 1.93660647), (376.34, 2.67382177), (376.4, 0.34587516), (374.89, 3.66), (376.4, 0.64543474), (376.4, 0.34587516), (376.39, 0.40600253), (376.26, 23.29955958), (376.39, 0.10644295), (376.26, 23.0), (376.27, 0.71587967), (500.0, 322.33761047), (500.0, 322.64552102), (376.47, 0.74308352), (376.27, 0.41632009), (376.3, 0.29955958), (379.27, 1.96007331), (376.43, 8.51701224), (374.5, 1.27976122)]
16786
[(372.85, 2.61805646), (372.84, 8.9237), (372.67, 8.0285), (372.66, 3.28509036), (372.6, 0.02326771), (372.25, 10.6798), (372.24, 6.493), (372.2, 2.0), (371.96, 6.692), (371.74, 0.04), (371.73, 7.265), (371.7, 0.02231636), (371.49, 6.928), (371.4, 0.02233438), (371.28, 7.441), (371.1, 0.02235244), (371.04, 6.84), (370.9, 0.262), (370.88, 2.16480323), (370.51, 51.0)]
3523


In [80]:
askside = {373.1: 0.53671, 
           373.23: 17.24,
           373.27: 7.9555,
           373.28: 1.3363,
           373.29: 0.11771, 
           373.5: 55.04631,
           373.55: 10.6796,
           373.56: 1.43616041, 
           373.59: 0.26566839, 
           373.6: 1.43882788,
           373.61: 1.43841224,
           373.65: 0.2630484, 
           373.71: 1.69908455, 
           373.73: 0.43790512, 
           373.27: 7.9555, 
           373.55: 10.6796, 
           373.22: 1.3363, 373.49: 7.9934, 373.72: 10.6566, 373.28: 1.3363}

asks = {373.27: 7.9929, 373.28: 0.0, 373.49: 0.0, 373.55: 10.6849, 373.72: 0.0}

for ask in asks:
    #print(asks.get(ask))
    if ask in askside:
        if asks.get(ask) == 0:
            askside.pop(ask)
        else:
            askside.pop(ask)
            askside.update({ask: asks.get(ask)})
    elif asks.get(ask) > 0:
        askside.update({ask: asks.get(ask)})
        if len(askside) > 20:
            askside.pop(max(askside))

            
pair = (1,2)
bidside = [(372.85,2.61805646),
           (372.84,8.9237),
           (372.67,8.0285),
           (372.66,3.28509036),
           (372.6,0.02326771),
           (372.25,10.6798),
           (372.24,6.493),
           (372.2,2.0),
           (371.96,6.692),
           (371.74,0.04),
           (371.73,7.265),
           (371.7,0.02231636),
           (371.49,6.928),
           (371.4,0.02233438),
           (371.28,7.441),
           (371.1,0.02235244),
           (371.04,6.84),
           (370.9,0.262),
           (370.88,2.16480323),
           (370.86,45.75)
]
bidside = dict(bidside)
print(bidside)

{371.1: 0.02235244, 371.49: 6.928, 370.86: 45.75, 371.7: 0.02231636, 372.2: 2.0, 371.73: 7.265, 370.88: 2.16480323, 372.25: 10.6798, 372.85: 2.61805646, 371.28: 7.441, 371.4: 0.02233438, 370.9: 0.262, 371.74: 0.04, 372.67: 8.0285, 371.96: 6.692, 372.24: 6.493, 372.66: 3.28509036, 372.84: 8.9237, 371.04: 6.84, 372.6: 0.02326771}


# Collect and transform order book states from CryptoIQ

In [4]:
URL = 'https://cryptoiq.io/api/marketdata/orderbooktop/bitstamp/btcusd/2016-%s'

dates = pd.date_range(start = '1/1/2016', end = '5/1/2016', freq='H')

lob_data = pd.DataFrame()

for date in dates:
    time = str(date.month) + '-' + str(date.day) + '/' + str(date.hour)
    data = pd.read_json(URL % time)
    lob_data = lob_data.append(data)

#lob_data.to_csv(path_or_buf='../btc-data/BTC_LOB_collected.csv')

# Convert indices to datetime format (rows: 939612)

In [5]:
lob_data.set_index('time', inplace=True)
lob_data.index = pd.to_datetime(lob_data.index)

# Create indices for evenly spaced time series (10s)

In [16]:
dates = pd.date_range(start = '1/1/2016 00:00:00', end = '5/1/2016 00:59:50', freq='10s')
dates
#lob_data.to_csv(path_or_buf='../btc-data/BTC_LOB_collected.csv')

DatetimeIndex(['2016-01-01 00:00:00', '2016-01-01 00:00:10',
               '2016-01-01 00:00:20', '2016-01-01 00:00:30',
               '2016-01-01 00:00:40', '2016-01-01 00:00:50',
               '2016-01-01 00:01:00', '2016-01-01 00:01:10',
               '2016-01-01 00:01:20', '2016-01-01 00:01:30',
               ...
               '2016-05-01 00:58:20', '2016-05-01 00:58:30',
               '2016-05-01 00:58:40', '2016-05-01 00:58:50',
               '2016-05-01 00:59:00', '2016-05-01 00:59:10',
               '2016-05-01 00:59:20', '2016-05-01 00:59:30',
               '2016-05-01 00:59:40', '2016-05-01 00:59:50'],
              dtype='datetime64[ns]', length=1045800, freq='10S')

In [7]:
lob_data['asks'] = lob_data['asks'].map(dict)
lob_data['bids'] = lob_data['bids'].map(dict)

# Re-index the LOB table with the evenly spaced time series, fill missing values with the nearest available prices (rows: 1045800) 

In [19]:
lob_data = lob_data.reindex(dates, method = 'nearest')

In [20]:
lob_data

,asks,bids
2016-01-01 00:00:00,"{432.52: 1.17, 432.36: 6.9631, 431.26: 1.13740...","{428.65: 0.02799486, 429.33: 39.75, 429.47: 32..."
2016-01-01 00:00:10,"{432.53: 1.1774200000000001, 432.48: 13.8492, ...","{428.65: 0.02799486, 429.38: 16.114, 429.35: 1..."
2016-01-01 00:00:20,"{432.59: 17.8, 432.52: 1.17, 431.26: 1.1374035...","{428.65: 0.02799486, 429.41: 9.2521, 429.33: 3..."
2016-01-01 00:00:30,"{432.6: 17.80005534, 432.36: 6.955600000000000...","{429.47: 6.9589, 428.33: 4.41, 429.37: 32.0333..."
2016-01-01 00:00:40,"{432.58: 40.92, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.28: 0.012, 428.93: 0...."
2016-01-01 00:00:50,"{432.58: 40.92, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.2: 39.73, 429.28: 0.0..."
2016-01-01 00:01:00,"{432.58: 40.92, 432.36: 6.9556000000000004, 43...","{428.65: 0.02799486, 429.45: 39.75, 429.01: 14..."
2016-01-01 00:01:10,"{432.58: 40.92, 432.35: 6.9594000000000005, 43...","{428.65: 0.02799486, 429.47: 14.916, 429.45: 3..."
2016-01-01 00:01:20,"{432.58: 40.92, 432.35: 6.8916, 431.26: 1.1374...","{428.65: 0.02799486, 429.47: 2.35, 429.91: 32...."
2016-01-01 00:01:30,"{432.58: 40.92, 432.37: 6.913, 431.26: 1.13740...","{428.65: 0.02799486, 429.45: 39.75, 429.01: 14..."


In [21]:
lob_data.to_csv(path_or_buf='../btc-data/BTC_LOB_collected.csv')

In [36]:
lob_features = pd.DataFrame(index = dates)
lob_features['ask volume'] = lob_data['asks'].map(lambda x: sum(x.values()))

In [39]:
lob_features['bid volume'] = lob_data['bids'].map(lambda x: sum(x.values()))
lob_features['ask price'] = lob_data['asks'].map(min)
lob_features['bid price'] = lob_data['bids'].map(max)

In [45]:
lob_features['bid-ask spread'] = lob_features['ask price'] - lob_features['bid price']
lob_features['mid price'] = (lob_features['ask price'] + lob_features['bid price'])/2
lob_features['ask price spread'] = lob_data['asks'].map(max) - lob_features['ask price']
lob_features['bid price spread'] = lob_features['bid price'] - lob_data['bids'].map(min)
lob_features['mean ask volume'] = lob_features['ask volume'] / 20
lob_features['mean bid volume'] = lob_features['bid volume'] / 20
lob_features['mean ask price'] = lob_data['asks'].map(sum) / 20
lob_features['mean bid price'] = lob_data['bids'].map(sum) / 20

,ask volume,bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:00:10,318.629099,323.428038,430.89,429.84,1.05,430.365,2.11,1.19,15.931455,16.171402,432.1875,429.1115
2016-01-01 00:00:20,290.919269,354.135003,430.89,429.86,1.03,430.375,2.44,1.54,14.545963,17.706750,432.2465,429.0100
2016-01-01 00:00:30,232.153855,172.031328,430.89,429.89,1.00,430.390,2.80,1.56,11.607693,8.601566,432.3350,429.0750
2016-01-01 00:00:40,290.952369,322.855038,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,16.142752,432.2360,429.1115
2016-01-01 00:00:50,290.952369,287.730638,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,14.386532,432.2360,429.1010
2016-01-01 00:01:00,290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:01:10,290.875538,322.244428,430.89,429.90,0.99,430.395,2.44,1.55,14.543777,16.112221,432.2355,429.0825
2016-01-01 00:01:20,279.887738,309.666428,430.89,429.91,0.98,430.400,2.32,1.56,13.994387,15.483321,432.2295,429.1055
2016-01-01 00:01:30,285.901138,307.380028,430.89,429.92,0.97,430.405,2.44,1.57,14.295057,15.369001,432.3270,429.0700


In [46]:
lob_features.rename(columns = {'ask volume':'total ask volume', 'bid volume':'total bid volume'}, inplace = True)
lob_features

,total ask volume,total bid volume,ask price,bid price,bid-ask spread,mid price,ask price spread,bid price spread,mean ask volume,mean bid volume,mean ask price,mean bid price
2016-01-01 00:00:00,318.632799,387.595528,430.89,429.84,1.05,430.365,2.11,1.52,15.931640,19.379776,432.1405,429.0090
2016-01-01 00:00:10,318.629099,323.428038,430.89,429.84,1.05,430.365,2.11,1.19,15.931455,16.171402,432.1875,429.1115
2016-01-01 00:00:20,290.919269,354.135003,430.89,429.86,1.03,430.375,2.44,1.54,14.545963,17.706750,432.2465,429.0100
2016-01-01 00:00:30,232.153855,172.031328,430.89,429.89,1.00,430.390,2.80,1.56,11.607693,8.601566,432.3350,429.0750
2016-01-01 00:00:40,290.952369,322.855038,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,16.142752,432.2360,429.1115
2016-01-01 00:00:50,290.952369,287.730638,430.89,429.90,0.99,430.395,2.44,1.25,14.547618,14.386532,432.2360,429.1010
2016-01-01 00:01:00,290.871738,374.632128,430.89,429.90,0.99,430.395,2.44,1.58,14.543587,18.731606,432.2360,428.9660
2016-01-01 00:01:10,290.875538,322.244428,430.89,429.90,0.99,430.395,2.44,1.55,14.543777,16.112221,432.2355,429.0825
2016-01-01 00:01:20,279.887738,309.666428,430.89,429.91,0.98,430.400,2.32,1.56,13.994387,15.483321,432.2295,429.1055
2016-01-01 00:01:30,285.901138,307.380028,430.89,429.92,0.97,430.405,2.44,1.57,14.295057,15.369001,432.3270,429.0700


In [47]:
lob_features.to_csv(path_or_buf='../btc-data/BTC_LOB_features_10s.csv')